In [49]:
import csv

In [66]:
class subject:
    
    def __init__(self,subno,subname,itp,crd):
        self.subno=subno
        self.subname=subname
        self.itp=itp
        self.crd=crd
        self.grade=''
        self.type=''

class sem:
    
    def __init__(self):
        self.spi=0
        self.courses=[]
        self.credits=0

class student:
    
    def __init__(self):
        self.roll=''
        self.name=''
        self.department=''
        self.cpi=[0 for i in range(8)]
        self.sem_data=list(None for i in range(8))
        self.sem_credits=[0 for i in range(8)]
        
        

file="names-roll.csv"
rn=dict()
with open(file, 'r') as csvfile:
    csvreader = (csv.reader(csvfile))
    for x in (csvreader):
        k=student()
        k.roll=x[0]
        k.name=x[1]
        rn[x[0]]=k
    del rn['Roll']
       

In [84]:
file="subjects_master.csv"
sub=dict()
with open(file, 'r') as csvfile:
    csvreader = (csv.reader(csvfile))
    next(csvreader)
    for x in csvreader:
        k=subject(x[0],x[1],x[2],int(x[3]))
        sub[x[0]]=k
    for x in sub:
        

In [81]:
file="grades.csv"
with open(file, 'r') as csvfile:
    csvreader = (csv.reader(csvfile))
    next(csvreader)
    for x in csvreader:
        if not rn[x[0]].sem_data[int(x[1])-1] :
            rn[x[0]].sem_data[int(x[1])-1]=sem()
        sd=rn[x[0]].sem_data[int(x[1])-1]
        sd_courses=sd.courses
        sd_courses.append(sub[x[2]])
        sd_courses[-1].grade=x[4]
        sd_courses[-1].type=x[5]
        if x[5][0]=='D':
            rn[x[0]].department=(sub[x[2]].subno)[0:2]
        
      

In [82]:
for x in rn:
    s=rn[x]
    print([s.roll,s.name,s.department,s.sem_credits])
        

['0401CS01', 'Marsiella Gourlay', 'CS', [204, 176, 172, 188, 168, 160, 164, 160]]
['0401CS02', 'Opaline Cristofol', 'CS', [196, 176, 172, 188, 168, 160, 164, 160]]
['0401CS03', 'Tonya Merrigan', 'CS', [196, 176, 172, 188, 168, 160, 164, 160]]
['0401CS04', 'Jillian Andren', 'CS', [196, 176, 172, 188, 168, 160, 164, 160]]
['0401CS05', 'Isiahi Sleith', 'CS', [196, 176, 129, 141, 126, 120, 123, 120]]
['0401CS06', 'Wake Culpin', 'CS', [147, 132, 129, 141, 126, 120, 123, 120]]
['0401CS07', 'Audrye Puckett', 'CS', [147, 132, 129, 141, 126, 120, 123, 120]]
['0401CS08', 'Major Thorald', 'CS', [147, 132, 129, 141, 126, 120, 123, 120]]
['0401CS09', 'Chery Stephens', 'CS', [147, 132, 129, 141, 126, 120, 123, 120]]
['0401CS10', 'Dan Knights', 'CS', [147, 132, 129, 141, 126, 120, 123, 120]]
['0401CS11', 'Boycie Gillow', 'CS', [147, 132, 129, 141, 126, 120, 123, 120]]
['0401CS12', 'Farrah Lowe', 'CS', [147, 132, 129, 141, 126, 120, 123, 120]]
['0401CS13', 'Taite Hussy', 'CS', [147, 132, 129, 141, 126